In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import preprocessing 
import sklearn

In [2]:
def ConvertToLabel(df,pos):
    label_encoder = preprocessing.LabelEncoder() 
    df[pos]= label_encoder.fit_transform(df[pos]) 
    return df

def chunks(df, n):
    for i in range(0, len(df), n):
        yield df[i:i + n]

### Data Import

In [3]:
X_1 = pd.read_csv("project3_dataset1.txt",header=None, delimiter="\t")
XShape, YShape = X_1.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))
Y_1 = X_1[YShape-1]
X_1 = X_1.iloc[:, :YShape-1]
X_1 = X_1.to_numpy()
Y_1 = Y_1.to_numpy()
X_1 = sklearn.preprocessing.normalize(X_1)

X Shape:569 Y Shape: 31


In [4]:
X_2 = pd.read_csv("project3_dataset2.txt",header=None, delimiter="\t")
XShape, YShape = X_2.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))

Y_2 = X_2[YShape-1]
X_2 = ConvertToLabel(X_2,4)
X_2 = X_2.iloc[:, :YShape-1]
X_2 = X_2.to_numpy()
Y_2 = Y_2.to_numpy()
X_2 = sklearn.preprocessing.normalize(X_2)

X Shape:462 Y Shape: 10


## Library Implementation

In [5]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.model_selection import train_test_split

Data Set 1

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_1, Y_1, test_size=0.2)

In [7]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8245614035087719


Data Set 2

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_2, Y_2, test_size=0.2)

In [9]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6129032258064516


## Scratch Implementation

In [10]:
def count_occ(value, index, lt):
    counter = 0
    for row in lt:
        if row[index] == value:
            counter+=1
    return counter

def findMean(train_x):
    return np.mean(train_x, axis=0)

def findStdDev(train_x):
    return np.std(train_x, axis=0)

def gaussian(v,mean,stdDev,index):
    return (1/(2 * 3.14 * (stdDev[index]**2))**0.5) * math.exp(-1 * ((v - mean[index])**2)/(2*(stdDev[index]**2)))

def naiveBayes(train_x,train_y,test_x):
    list_yes,list_no = [],[]
    dict_yes,dict_no = {},{}
    output = []
    for i,items in enumerate(train_y):
        if train_y[i] == 0.0:
            list_no.append(train_x[i])
        else:
            list_yes.append(train_x[i])
    mean1, stdDev1 = findMean(list_yes), findStdDev(list_yes)
    mean2, stdDev2 = findMean(list_no), findStdDev(list_no)
    for items in list_yes:
        for index,item in enumerate(items):
            if (index,item) not in dict_yes:
                dict_yes[(index, item)] = gaussian(item, mean1, stdDev1, index)
                
    for items in list_no:
        for index,item in enumerate(items):
            if (index,item) not in dict_no:
                dict_yes[(index, item)] = gaussian(item, mean2, stdDev2, index)

    for item in test_x:
        product_yes,product_no = 1.0, 1.0
        for index, i in enumerate(item):
            product_yes = product_yes * gaussian(i, mean1, stdDev1, index)
            product_no = product_no * gaussian(i, mean2, stdDev2, index)
      
        product_yes = (product_yes) * (len(list_yes)/len(train_y))
        product_no = (product_no) * (len(list_no)/len(train_y))

        if product_yes >= product_no:
            output.append(1.0)
        else:
            output.append(0.0)    
    return output

In [11]:
def findPredictedClass(predicted_labels,test_y):
    tp,tn,fp,fn = 0,0,0,0
    for i in range(len(predicted_labels)):
        if predicted_labels[i] == 1 and test_y[i] == 1:
            tp+=1    
        if predicted_labels[i] == 0 and test_y[i] == 0:
            tn+=1
        if predicted_labels[i] == 1 and test_y[i] == 0:
            fp+=1
        if predicted_labels[i] == 0 and test_y[i] == 1:
            fn+=1
    try:
        accuracy = (tp + tn)/(tp + tn + fp + fn)
    except:
        accuracy = 0.0
    try:
        precision = tp / (tp + fp)
    except:
        precision = 0.0
    try:
        recall = tp / (tp + fn)
    except:
        recall = 0.0
    try:
        fscore = 2 * (precision * recall)/(precision + recall)
    except:
        fscore = 0.0
    return accuracy,precision,recall,fscore

In [12]:
def dataProcessing(X,Y):
    data = np.column_stack((X, Y))
    np.random.shuffle(data)
    RowCount = math.ceil(len(X)/10)
    output = list(chunks(data,RowCount))
    print('Output Count:'+str(len(output))+' Output X Shape:'+str(len(output[0]))+' Output Y Shape:'+str(len(output[0][0])))
    return output

In [13]:
def knnNaiveBayes(output):
    totalAccuracy, totalPrecision, totalRecall, totalFscore = [],[],[],[]
    for i in range(len(output)):
        test_x,test_y,train_x, train_y = [], [], [], []
        train_x = [row for j,lt in enumerate(output) if j!=i for row in lt]
        test_x = output[i]
        for arr in test_x:
            test_y.append(arr[-1])
        for arr in train_x:
            train_y.append(arr[-1])
        train_x = np.delete(train_x, np.s_[-1:], axis=1)
        test_x = np.delete(test_x, np.s_[-1:], axis=1)
        
        predicted_labels = naiveBayes(train_x,train_y,test_x)
        accuracy, precision, recall, fscore = findPredictedClass(predicted_labels,test_y)
        totalAccuracy.append(accuracy)
        totalPrecision.append(precision)
        totalRecall.append(recall)
        totalFscore.append(fscore)

    avgAccuracy  = np.mean(totalAccuracy)
    avgPrecision = np.mean(totalPrecision)
    avgRecall    = np.mean(totalRecall)
    avgFscore    = np.mean(totalFscore)    
    print('Avgerage Accuracy :'+ str(avgAccuracy))
    print('Avgerage Precision :'+ str(avgPrecision))
    print('Avgerage Recall :'+ str(avgRecall))
    print('Avgerage Fscore :'+ str(avgFscore))

In [15]:
output = dataProcessing(X_1, Y_1)
knnNaiveBayes(output)

Output Count:10 Output X Shape:57 Output Y Shape:31
Avgerage Accuracy :0.8118421052631579
Avgerage Precision :0.6921608021910647
Avgerage Recall :0.9101193968756898
Avgerage Fscore :0.7820636944573826


In [16]:
output_2 = dataProcessing(X_2, Y_2)
knnNaiveBayes(output_2)

Output Count:10 Output X Shape:47 Output Y Shape:10
Avgerage Accuracy :0.6921985815602837
Avgerage Precision :0.567946571047594
Avgerage Recall :0.5414020344980097
Avgerage Fscore :0.5386643540272573


## Data Set 3

In [17]:
TrainData = pd.read_csv("project3_dataset3_train.txt",header=None, delimiter="\t")
XShape, YShape = TrainData.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))
TrainData_Y = TrainData[YShape-1]
TrainData_X = TrainData.iloc[:, :YShape-1]
TrainData_X = TrainData_X.to_numpy()
TrainData_Y = TrainData_Y.to_numpy()

X Shape:80 Y Shape: 5


In [18]:
TestData = pd.read_csv("project3_dataset3_test.txt",header=None, delimiter="\t")
XShape, YShape = TestData.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))
TestData_Y = TestData[YShape-1]
TestData_X = TestData.iloc[:, :YShape-1]
TestData_X = TestData_X.to_numpy()
TestData_Y = TestData_Y.to_numpy()

X Shape:20 Y Shape: 5


In [19]:
predicted_labels = naiveBayes(TrainData_X, TrainData_Y, TestData_X)
accuracy, precision, recall, fscore = findPredictedClass(predicted_labels,TestData_Y)
print(' Accuracy :'+ str(accuracy))
print(' Precision :'+ str(precision))
print(' Recall :'+ str(recall))
print(' Fscore :'+ str(fscore))

 Accuracy :0.95
 Precision :0.9090909090909091
 Recall :1.0
 Fscore :0.9523809523809523


## Data Set 4

In [20]:
Data4 = pd.read_csv("project3_dataset4.txt",header=None, delimiter="\t")

In [21]:
for i in range(0,4):
    Data4 = ConvertToLabel(Data4,i)

In [22]:
XShape, YShape = Data4.shape
print('X Shape:' + str(XShape) + ' Y Shape: '+ str(YShape))
Data4_Y = Data4[YShape-1]
Data4_X = Data4.iloc[:, :YShape-1]
Data4_X = Data4_X.to_numpy()
Data4_Y = Data4_Y.to_numpy()

X Shape:14 Y Shape: 5


In [23]:
X_train, X_test, y_train, y_test = train_test_split(Data4_X, Data4_Y, test_size=0.3)

In [25]:
predicted_labels = naiveBayes(X_train, y_train, X_test)
accuracy, precision, recall, fscore = findPredictedClass(predicted_labels,y_test)
print(' Accuracy :'+ str(accuracy))
print(' Precision :'+ str(precision))
print(' Recall :'+ str(recall))
print(' Fscore :'+ str(fscore))

 Accuracy :0.8
 Precision :1.0
 Recall :0.5
 Fscore :0.6666666666666666


## As Requested for Demo

In [26]:
Data4 = pd.read_csv("project3_dataset4.txt",header=None, delimiter="\t")
Data4 = Data4.to_numpy()

In [27]:
def demo(dataset,testdata):

    def getindex(dataset):
        index = []
        for i in range(dataset.shape[1]):
            try:
                float(dataset[0,i])
            except ValueError:
                index.append(i)
        return index
    
    traindata = dataset
    index = getindex(dataset)

    labels = list(dataset[:,-1])

    prior_0 = labels.count(0)/len(labels)
    prior_1 = labels.count(1)/len(labels)

    denominator = 1
    for i in range(len(testdata)):
        val = list(dataset[:,i])
        num = val.count(testdata[i])
        den = num/len(dataset)
        denominator = denominator*den

    post_0 = 1
    post_1 = 1
    for i in range(len(index)):
        train_data = traindata[:,index[i]]
        label_train = traindata[:, -1]
        dict = {}
        for j in range(len(label_train)):
            if label_train[j] not in dict:
                dict[label_train[j]] = []
            dict[label_train[j]].append(train_data[j])
        posterior_0 = dict[0].count(testdata[i])/len(dict[0])
        posterior_1 = dict[1].count(testdata[i]) / len(dict[1])
        post_0 = post_0 * posterior_0
        post_1 = post_1 * posterior_1

    prob_list = []
    prob_0 = (post_0*prior_0)/denominator
    prob_1 = (post_1*prior_1)/denominator
    prob_list.append(prob_0)
    prob_list.append(prob_1)
    val = prob_list.index(max(prob_list))
    print(post_0)
    print(post_1)
    print(prior_0)
    print(prior_1)
    #print(post_0*prior_0)
    #print(post_1*prior_1)
    print("probability for class 0 : ", prob_0/(prob_0+prob_1))
    print("probability for class 1 : ", prob_1/(prob_0+prob_1))
    print("The input will get classified to class : ", val)

    return prob_0,prob_1,val

In [28]:
testdata = ['sunny', 'cool', 'high', 'weak']
demo(Data4,testdata)

0.038400000000000004
0.016460905349794237
0.35714285714285715
0.6428571428571429
probability for class 0 :  0.5644599303135889
probability for class 1 :  0.4355400696864111
The input will get classified to class :  0


(0.47040000000000004, 0.36296296296296293, 0)